In [63]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
import pandas as pd
from glob import glob 
from fastcore.xtras import load_pickle

from gpt3forchem.output import get_regression_metrics
from gpt3forchem.api_wrappers import extract_prediction

from pycm import ConfusionMatrix

## Classification

In [65]:
all_res = glob('results/20221130_freesolv/*')

In [66]:
metrics = []
baselines = []

for res in all_res:
    res = load_pickle(res)
    cm = res['cm']
    cm = ConfusionMatrix(list(map(lambda x: str(x).strip(), res['cm'].actual_vector)), list(map(lambda x: str(x).strip(), res['cm'].predict_vector)))
    baseline_cm = res['baseline']['cm']
    baseline_cm = ConfusionMatrix(list(map(lambda x: str(x).strip(), baseline_cm.actual_vector)), list(map(lambda x: str(x).strip(), baseline_cm.predict_vector)))
    metrics.append(
        {
            'train_size': res['train_size'],
            'representation': res['representation'],
            'accuracy': cm.ACC_Macro,
            'f1_macro': cm.F1_Macro,
            'f1_micro': cm.F1_Micro
        })
    baselines.append(
        {
            'train_size': res['train_size'],
            'representation': res['representation'],
            'accuracy': baseline_cm.ACC_Macro,
            'f1_macro': baseline_cm.F1_Macro,
            'f1_micro': baseline_cm.F1_Micro  
        })

In [67]:
metrics = pd.DataFrame(metrics)
baselines = pd.DataFrame(baselines)

In [76]:
metrics.groupby(['representation', 'train_size']).agg(['mean', 'std', 'count'])

accuracy                  f1_macro                  \
                               mean       std count      mean       std count   
representation train_size                                                       
inchi          10          0.787000  0.032527     2  0.173926  0.003525     2   
               50          0.833000  0.007071     2  0.222916  0.052498     2   
               500         0.933333  0.002857     2  0.833803  0.014607     2   
iupac_name     10          0.802000  0.036770     2  0.186780  0.007602     2   
               50          0.861000  0.029698     2  0.368536  0.147473     2   
               500         0.933333  0.014285     2  0.759600  0.123082     2   
selfies        10          0.772000  0.039598     2  0.160942  0.068168     2   
               50          0.873000  0.012728     2  0.284819  0.014746     2   
               500         0.921212  0.014285     2  0.831440  0.047642     2   
smiles         10          0.841500  0.008699     4  0.249858  0.015695     4   
               50          0.868000  0.011314     2  0.315538  0.035795     2   
               500         0.945455  0.002857     2  0.873669  0.003355     2   

                           f1_micro                  
                               mean       std count  
representation train_size                            
inchi          10          0.467500  0.081317     2  
               50          0.582500  0.017678     2  
               500         0.833333  0.007142     2  
iupac_name     10          0.505000  0.091924     2  
               50          0.652500  0.074246     2  
               500         0.833333  0.035712     2  
selfies        10          0.430000  0.098995     2  
               50          0.682500  0.031820     2  
               500         0.803030  0.035712     2  
smiles         10          0.603750  0.021747     4  
               50          0.670000  0.028284     2  
               500         0.863636  0.007142     2

In [70]:
baselines.groupby(['train_size']).agg(['mean', 'std'])

accuracy            f1_macro           f1_micro          
               mean       std      mean       std     mean       std
train_size                                                          
10           0.8316  0.000843  0.233844  0.026455    0.579  0.002108
50           0.8940  0.000000  0.437840  0.000000    0.735  0.000000
500          0.8000  0.000000  0.000000  0.000000    0.000  0.000000

## Regression

In [71]:
all_res_regression = glob('results/20221129_freesolv_regression/*')

In [72]:
metrics_regression = []
baselines_regression = []

for res in all_res_regression:
    res = load_pickle(res)
    metrics_regression.append(
        {
            'train_size': res['train_size'],
            'representation': res['representation'],
            'r2': res['metrics']['r2'],
            'max_error': res['metrics']['max_error'],
            'mean_absolute_error': res['metrics']['mean_absolute_error'],
            'mean_squared_error': res['metrics']['mean_squared_error'],
            'rmse': res['metrics']['rmse'],
        })
    baselines_regression.append(
        {
            'train_size': res['train_size'],
            'representation': res['representation'],
            'r2': res['baseline']['r2'],
            'max_error': res['baseline']['max_error'],
            'mean_absolute_error': res['baseline']['mean_absolute_error'], 
            'mean_squared_error': res['baseline']['mean_squared_error'],
            'rmse': res['baseline']['rmse'],
        })

In [73]:
metrics_regression = pd.DataFrame(metrics_regression)

baselines_regression = pd.DataFrame(baselines_regression)

In [74]:
metrics_regression.groupby(['representation', 'train_size']).agg(['mean', 'std'])

r2            max_error            \
                               mean       std       mean       std   
representation train_size                                            
inchi          10         -0.245881  0.053283  18.935000  2.241528   
               50          0.010655  0.062585  17.860000  0.169706   
               500         0.651623  0.035518  13.955000  0.106066   
iupac_name     10         -0.418369  0.108768  17.990000  0.579828   
               50          0.082566  0.170367  17.740000  0.000000   
               500         0.773709  0.004334   7.695000  1.180868   
selfies        10         -0.030135  0.118407  17.780000  0.399500   
               50          0.164391  0.086608  16.196667  2.875036   
               500         0.764539  0.012558  10.065000  1.491995   
smiles         10         -0.178439  0.169525  19.046667  2.051747   
               50          0.087600  0.119503  18.512000  1.252745   
               500         0.817345  0.071616   9.096667  4.790035   

                          mean_absolute_error           mean_squared_error  \
                                         mean       std               mean   
representation train_size                                                    
inchi          10                    3.274850  0.088106          19.134028   
               50                    2.858700  0.046457          14.674768   
               500                   1.378434  0.065925           5.380518   
iupac_name     10                    3.686525  0.141103          21.783073   
               50                    2.765350  0.300167          13.608130   
               500                   1.122172  0.011214           3.494948   
selfies        10                    2.902700  0.143880          15.625310   
               50                    2.647633  0.178014          12.223212   
               500                   1.137525  0.023213           3.636576   
smiles         10                    3.189067  0.337366          17.906469   
               50                    2.683390  0.160492          13.297929   
               500                   0.985185  0.071981           2.928677   

                                         rmse            
                                std      mean       std  
representation train_size                                
inchi          10          0.818318  4.373746  0.093549  
               50          0.928305  3.829807  0.121195  
               500         0.548563  2.318085  0.118322  
iupac_name     10          1.670438  4.665517  0.179020  
               50          2.527027  3.680926  0.343260  
               500         0.066934  1.869435  0.017902  
selfies        10          1.512037  3.949803  0.191173  
               50          1.127590  3.493620  0.163538  
               500         0.193952  1.906642  0.050862  
smiles         10          2.677896  4.222078  0.310857  
               50          1.481470  3.642259  0.199612  
               500         1.286264  1.685005  0.366267

In [75]:
baselines_regression.groupby(['train_size']).agg(['mean', 'std'])

r2            max_error           mean_absolute_error  \
                mean       std       mean       std                mean   
train_size                                                                
10          0.139089  0.027464  20.501197  0.071450            2.730168   
50          0.709372  0.008226   9.432677  0.606000            1.582631   
500         0.916150  0.006269   3.954292  0.296657            0.868814   

                     mean_squared_error                rmse            
                 std               mean       std      mean       std  
train_size                                                             
10          0.041038          13.914439  0.149266  3.730157  0.019943  
50          0.014952           4.577601  0.001755  2.139533  0.000410  
500         0.036917           1.379811  0.134440  1.173245  0.060993